#                              CREDIT ANALYTICS

## Importing libraries/packages

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import db_dtypes

## CREATE CONNECTION TO BIGQUERY

In [ ]:

#Credentials and Client
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

# Display the client details
client

## READ AND LOAD DATA FROM BIGQUERY

In [ ]:
query = "SELECT * FROM `Credit_Business.Disbursements`"
query_job = client.query(query)

## CREATE PANDAS DATA FRAME 

In [ ]:
#Converting the query result into pandas dataframe
df = query_job.to_dataframe()

## DATA WRANGLING

In [ ]:
#SETTING PANDAS DISPLAY OPTIONS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.2f}'.format

#DROP COLUMNS
columns_to_drop = ['Branch_Code', 'Account_Number', 'Account_Name', 'S_N', 'GROUP___CUSTOMER_NUMBER']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# CONVERTING DATATYPES
df = df.astype({
    'Disbursed_Date': 'datetime64',
    'Disbursed_By': 'string',
    'Approved_By': 'string',
    'Branch_Name': 'string',
    'Insurance_company': 'string',
    'Sub_Sector': 'string',
    'Birth_date': 'datetime64',
    'Due_days': 'int64',
    'Interest_Rate': 'float64',
    'Principal_Outstanding': 'float64',
    'Interest_Outstanding': 'float64',
    'Grace_Period': 'string',
    'Loan_Cycle': 'int64',
    'Term': 'string',
    'Gender': 'string',
    'Product': 'string',
    'Sector': 'string',
    'Disbursed___Amount': 'float64',   
})

#DATAFRAME HEAD AND TAIL
df.head(10)
#df.tail(10)


,Branch_Name,Gender,Birth_date,Product,Sector,Loan_Cycle,Term,Disbursed___Amount,Interest_Rate,Principal_Outstanding,Interest_Outstanding,Grace_Period,Due_days,Disbursed_Date,Disbursed_By,Approved_By,Insurance_company,Sub_Sector
0,Head Office,M,1987-08-08,Staff Salary Advance,Trade,8,6 M,"1,000,000.00",0.00,"1,000,000.00",0.00,<NA>,0,2023-11-10,ODONG ALEX,Yahaya Ochaya,<NA>,Exports
1,Head Office,M,1992-05-02,Staff Salary Advance,Personal Loans and Household Loans,8,6 M,"2,700,000.00",0.00,"2,700,000.00",0.00,<NA>,0,2023-11-07,ODONG ALEX,Solomon Mwesigwa,<NA>,Personal Loans and Household Loans
2,Head Office,M,1987-02-07,Staff Salary Advance,Personal Loans and Household Loans,3,6 M,"4,000,000.00",0.00,"4,000,000.00",0.00,<NA>,0,2023-11-06,ODONG ALEX,Henry Bukenya,<NA>,Personal Loans and Household Loans
3,Head Office,F,1990-03-03,Staff Salary Advance,Personal Loans and Household Loans,4,6 M,"7,500,000.00",0.00,"7,500,000.00",0.00,<NA>,0,2023-11-01,NAMIREMBE MILLY,Jimmy Kasoro,<NA>,Personal Loans and Household Loans
4,Head Office,M,1990-04-10,Staff Salary Advance,Personal Loans and Household Loans,2,6 M,"3,100,000.00",0.00,"3,100,000.00",0.00,<NA>,0,2023-11-10,ODONG ALEX,Yahaya Ochaya,<NA>,Personal Loans and Household Loans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91594,Kabwohe,M,1997-05-22,Community Banking Loans,Trade,1,40 W,"1,000,000.00",0.29,"649,653.80","139,423.12",1 W,0,2023-07-14,ATURIHIHI ADSON,Sarah Kakande,Liberty Life Insurance,Exports
91595,Kabwohe,F,1957-12-23,Community Banking Loans,Trade,1,40 W,"500,000.00",0.18,"487,500.00","67,230.77",1 W,0,2023-10-13,BISHERURWA CONRAD,Robert Kiwanuka,Liberty Life Insurance,Exports
91596,Kabwohe,F,1996-09-09,Community Banking Loans,Trade,1,40 W,"500,000.00",0.29,"310,615.36","66,923.10",1 W,0,2023-07-07,ATURIHIHI ADSON,Sarah Kakande,Liberty Life Insurance,Exports
91597,Kabwohe,M,1967-02-17,Community Banking Loans,Trade,1,40 W,"1,000,000.00",0.29,"624,230.72","133,846.20",1 W,0,2023-07-07,ATURIHIHI ADSON,Sarah Kakande,Liberty Life Insurance,Exports


In [ ]:
#DATA FRAME COLUMNS
#print(df.columns)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=34c99b87-7747-401d-b203-ddc5eb2646f3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>